In [1]:
import pandas as pd
import re
df_neg = pd.read_csv('/home/yash/Downloads/to_push/NLP-Models-master/Text-Classification/data/negative/negative',sep=",",header=None ,names=['sentence','type'])
df_neg['type']='negative'
df_pos = pd.read_csv('/home/yash/Downloads/to_push/NLP-Models-master/Text-Classification/data/positive/positive',sep=",",header=None ,names=['sentence','type'])
df_pos['type']='positive'
df=pd.concat([df_neg,df_pos])


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
MAX_NB_WORDS = 16384
MAX_SEQUENCE_LENGTH=128

In [4]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(df['sentence'].values)
X = tokenizer.texts_to_sequences(df['sentence'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [5]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = df['type']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)

x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2)

Found 20464 unique tokens.
('Shape of data tensor:', (10662, 128))
('Shape of label tensor:', (10662, 2))


In [6]:
embeddings_index = {}

f = open('/home/yash/glove.42B.300d.txt')
for line in f:
    #print line
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [8]:
emb_dimension=300
embedding_matrix = np.zeros((len(word_index) + 1, emb_dimension))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [9]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            emb_dimension,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [13]:
from keras.callbacks import History 
from keras import regularizers
from keras import optimizers
history = History()
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(256,return_sequences=True, dropout=0.4, recurrent_dropout=0.4, activation='relu',kernel_regularizer=regularizers.l2(0.001)))#,activity_regularizer=regularizers.l1(0.01)))
model.add(LSTM(256, dropout=0.4, recurrent_dropout=0.4, activation='relu',kernel_regularizer=regularizers.l2(0.001)))#,activity_regularizer=regularizers.l1(0.01)))

#model.add(LSTM(256, dropout_U=0.5, dropout_W=0.5, activation='relu', kernel_regularizer=l2_reg))
model.add(Dense(2,activation='softmax'))#,kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
rmsprop = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop,metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 300)          6139500   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128, 256)          570368    
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 7,235,694
Trainable params: 1,096,194
Non-trainable params: 6,139,500
_________________________________________________________________


In [14]:
history=model.fit(x_train, y_train,
          batch_size=512,
          epochs=300,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 8529 samples, validate on 2133 samples
Epoch 1/300
8529/8529 [==============================] - 15s 2ms/step - loss: 1.5449 - acc: 0.5160 - val_loss: 1.5112 - val_acc: 0.5218
Epoch 2/300
8529/8529 [==============================] - 13s 1ms/step - loss: 1.4855 - acc: 0.5567 - val_loss: 1.4537 - val_acc: 0.6240
Epoch 3/300
8529/8529 [==============================] - 13s 1ms/step - loss: 1.4289 - acc: 0.5839 - val_loss: 1.3972 - val_acc: 0.6554
Epoch 4/300
8529/8529 [==============================] - 13s 2ms/step - loss: 1.3760 - acc: 0.5930 - val_loss: 1.3397 - val_acc: 0.6709
Epoch 5/300
8529/8529 [==============================] - 13s 2ms/step - loss: 1.3164 - acc: 0.6344 - val_loss: 1.2774 - val_acc: 0.6793
Epoch 6/300
8529/8529 [==============================] - 13s 2ms/step - loss: 1.2634 - acc: 0.6518 - val_loss: 1.2146 - val_acc: 0.6957
Epoch 7/300
8529/8529 [==============================] - 13s 1ms/step - loss: 1.2034 - acc: 0.6824 - val_loss: 1.1581 - val_acc: 0.7196


8529/8529 [==============================] - 13s 2ms/step - loss: 0.5647 - acc: 0.7798 - val_loss: 0.5583 - val_acc: 0.7937
Epoch 61/300
8529/8529 [==============================] - 14s 2ms/step - loss: 0.5612 - acc: 0.7820 - val_loss: 0.5541 - val_acc: 0.7937
Epoch 62/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.5496 - acc: 0.7919 - val_loss: 0.5511 - val_acc: 0.7951
Epoch 63/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.5588 - acc: 0.7846 - val_loss: 0.5525 - val_acc: 0.7909
Epoch 64/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.5489 - acc: 0.7873 - val_loss: 0.5495 - val_acc: 0.7961
Epoch 65/300
8529/8529 [==============================] - 13s 1ms/step - loss: 0.5511 - acc: 0.7872 - val_loss: 0.5486 - val_acc: 0.7937
Epoch 66/300
8529/8529 [==============================] - 13s 1ms/step - loss: 0.5470 - acc: 0.7877 - val_loss: 0.5439 - val_acc: 0.7975
Epoch 67/300
8529/8529 [==============================

Epoch 120/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.4881 - acc: 0.8068 - val_loss: 0.5007 - val_acc: 0.8031
Epoch 121/300
8529/8529 [==============================] - 13s 1ms/step - loss: 0.4826 - acc: 0.8115 - val_loss: 0.5009 - val_acc: 0.8092
Epoch 122/300
8529/8529 [==============================] - 13s 1ms/step - loss: 0.4830 - acc: 0.8096 - val_loss: 0.4985 - val_acc: 0.8059
Epoch 123/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.4810 - acc: 0.8103 - val_loss: 0.4973 - val_acc: 0.8008
Epoch 124/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.4844 - acc: 0.8132 - val_loss: 0.4970 - val_acc: 0.8040
Epoch 125/300
8529/8529 [==============================] - 13s 1ms/step - loss: 0.4828 - acc: 0.8097 - val_loss: 0.4949 - val_acc: 0.8040
Epoch 126/300
8529/8529 [==============================] - 13s 2ms/step - loss: 0.4832 - acc: 0.8115 - val_loss: 0.4969 - val_acc: 0.8017
Epoch 127/300
8529/8529 [=========

KeyboardInterrupt: 

In [ ]:

import matplotlib.pyplot as plt

%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
example = tokenizer.texts_to_sequences(["food was hot "])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)
le.inverse_transform(np.argmax(model.predict(example)))

In [ ]:
# evaluate the model of test data

from sklearn.metrics import classification_report
predictions = model.predict(x_test)
#print(classification_report(predictions,y_test))
print("Accuracy :",model.evaluate(x_test,y_test))